In [8]:
import cv2
import numpy as np

In [9]:
image = cv2.imread(f"C:/Users/ASUS/Downloads/CamScanner/sample.jpg")
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.bilateralFilter(image_gray, d=9, sigmaColor=75, sigmaSpace=75)
edges = cv2.Canny(blurred, threshold1=100, threshold2=200)

contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
epsilon = 0.02 * cv2.arcLength(largest_contour, True)
approx = cv2.approxPolyDP(largest_contour, epsilon, True)

In [10]:
def order_points(pts):
    # مرتب‌سازی نقاط به صورت: top-left, top-right, bottom-right, bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # بالا چپ
    rect[2] = pts[np.argmax(s)]  # پایین راست

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # بالا راست
    rect[3] = pts[np.argmax(diff)]  # پایین چپ
    
    return rect

In [11]:
if len(approx) == 4:
    pts = approx.reshape(4, 2)
    rect = order_points(pts)

    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = int(max(widthA, widthB))

    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = int(max(heightA, heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
else:
    print("کانتور پیدا شده چهارضلعی نبود.")

In [12]:
window_name = "Document Filter Preview"
button_height = 50
width = warped.shape[1]
height = warped.shape[0] + button_height

filter_mode = 0
save_requested = False

buttons = [
    ("Original", 0),
    ("Gray", 1),
    ("B&W", 2),
    ("Magic Color", 3),
    ("Save", 4)
]

In [13]:
filter_mode = 0

def apply_filter(mode):
    if mode == 0:
        return warped
    elif mode == 1:
        return cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    elif mode == 2:
        gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        return cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 10)
    elif mode == 3:
        filtered = cv2.bilateralFilter(warped, 9, 75, 75)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(filtered, -1, sharpen_kernel)
        hsv = cv2.cvtColor(sharpened, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        s = np.clip(cv2.multiply(s, 1.3), 0, 255).astype(np.uint8)
        v = np.clip(cv2.multiply(v, 1.2), 0, 255).astype(np.uint8)
        enhanced_hsv = cv2.merge([h, s, v])
        return cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
    else:
        return warped


# ------------------ UI رسم ------------------
def draw_buttons(canvas):
    for label, idx in buttons:
        # Calculate button position
        x_start = int(idx * (width / len(buttons)))
        x_end = int((idx + 1) * (width / len(buttons)))
        cv2.rectangle(canvas, (x_start, height - button_height), (x_end, height), (30, 30, 30), -1)

        # Get text size for centering
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
        text_x = x_start + (x_end - x_start - text_size[0]) // 2
        text_y = height - (button_height - text_size[1]) // 2

        # Draw text centered in button
        cv2.putText(canvas, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)


def get_button_index(x):
    segment_width = int(width / len(buttons))
    return int(x // segment_width)


# ------------------ ماوس هندلر ------------------
def mouse_callback(event, x, y, flags, param):
    global filter_mode, save_requested  # ✅ Add this line
    if event == cv2.EVENT_LBUTTONDOWN and y >= height - button_height:
        idx = get_button_index(x)
        if idx == len(buttons) - 1:  # Save button
            save_requested = True
        else:
            filter_mode = idx


# ------------------ حلقه‌ی اصلی ------------------
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, mouse_callback)

while True:
    output = apply_filter(filter_mode)

    # Convert grayscale to BGR for display
    if len(output.shape) == 2:
        display_output = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR)
    else:
        display_output = output.copy()

    canvas = np.zeros((height, width, 3), dtype=np.uint8)
    canvas[:warped.shape[0], :, :] = display_output
    draw_buttons(canvas)

    cv2.imshow(window_name, canvas)

    # Save image if requested
    if save_requested:
        filename = f"output_filter_{filter_mode}.jpg"
        cv2.imwrite(filename, output)
        print(f"[✅] Image saved as {filename}")
        save_requested = False

    key = cv2.waitKey(1)
    if key == ord('q') or key == 27:  # Press Q or ESC to quit
        break

cv2.destroyAllWindows()


[✅] Image saved as output_filter_0.jpg
[✅] Image saved as output_filter_1.jpg
[✅] Image saved as output_filter_2.jpg
